In [263]:
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Activation, Dense, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [264]:
PATH = 'images/boxes'
INPUT_SIZE = (224, 224, 3)

In [265]:
with open("categories.txt", "r") as file:
    categories = file.read().splitlines()
    
num_classes = len(categories)

In [266]:
dataset = []
X = []
y = []

i = 0
for folder in categories:
    files = os.listdir(f'{PATH}/{categories[i]}')
    for file in files:
        img = cv.imread(f'{PATH}/{categories[i]}/{file}')
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        img = cv.resize(img, INPUT_SIZE[:2])
        dataset.append([img, i])
    i += 1

In [267]:
for images, labels in dataset:
    X.append(images)
    y.append(labels)

In [268]:
X = np.array(X).reshape(-1, INPUT_SIZE[0], INPUT_SIZE[1], INPUT_SIZE[2])
X = X / 255

In [269]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
)

In [270]:
y_cat_train = to_categorical(y_train, num_classes)
y_cat_test = to_categorical(y_test, num_classes)

In [305]:
epochs = 10
lr = 1e-4
batch_size = X_train.shape[0] // epochs

print(batch_size)

input = tf.keras.layers.Input(INPUT_SIZE)

conv_1 = Conv2D(filters=32, kernel_size=(3,3), activation='relu')(input)
pool_1 = MaxPool2D(pool_size=(2,2))(conv_1)

conv_2 = Conv2D(filters=64, kernel_size=(3,3), activation='relu')(pool_1)
pool_2 = MaxPool2D(pool_size=(2,2))(conv_2)

flatten = Flatten()(pool_2)

dense = Dense(128)(flatten)
dropout = Dropout(0.2)(dense)

output = Dense(num_classes, activation="sigmoid")(dropout)

model = Model(inputs=[input], outputs=[output])
model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=lr, decay=lr / epochs), metrics=["accuracy"])

9


In [307]:
model.fit(X_train, y_cat_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[])

Train on 86 samples, validate on 10 samples
Epoch 1/10
86/86 [==============================] - 7s 85ms/sample - loss: 1.3721 - accuracy: 0.4884 - val_loss: 0.6891 - val_accuracy: 0.5000
Epoch 2/10
86/86 [==============================] - 6s 67ms/sample - loss: 0.5904 - accuracy: 0.6802 - val_loss: 0.5340 - val_accuracy: 0.7500
Epoch 3/10
86/86 [==============================] - 6s 67ms/sample - loss: 0.4326 - accuracy: 0.8140 - val_loss: 0.2657 - val_accuracy: 0.9500
Epoch 4/10
86/86 [==============================] - 6s 74ms/sample - loss: 0.4145 - accuracy: 0.8256 - val_loss: 0.2647 - val_accuracy: 0.8500
Epoch 5/10
86/86 [==============================] - 6s 68ms/sample - loss: 0.4604 - accuracy: 0.7791 - val_loss: 0.2019 - val_accuracy: 0.9000
Epoch 6/10
86/86 [==============================] - 7s 77ms/sample - loss: 0.2473 - accuracy: 0.9186 - val_loss: 0.2142 - val_accuracy: 0.8000
Epoch 7/10
86/86 [==============================] - 6s 70ms/sample - loss: 0.1863 - accuracy: 0.94

In [308]:
loss, acc = model.evaluate(X_test, y_cat_test)
print("LOSS: {}, ACC: {}".format(loss, acc * 100))

11/11 [==============================] - 0s 21ms/sample - loss: 0.2209 - accuracy: 0.8182
LOSS: 0.2208608239889145, ACC: 81.81818127632141


In [309]:
pred = model.predict(X_test)

In [310]:
img = X_test[11]
plt.imshow(img)
img = img.reshape(-1, INPUT_SIZE[0], INPUT_SIZE[1], INPUT_SIZE[2])
model.predict(img)[0].argmax()

IndexError: index 11 is out of bounds for axis 0 with size 11

In [311]:
model.save('models/emptybox.h5')

In [312]:
img = cv.imread('images/boxes/test/6.png', 1)
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
img.shape
img = cv.resize(img, (224,224))
img = img / 255
plt.imshow(img)
img = img.reshape(-1, INPUT_SIZE[0], INPUT_SIZE[1], INPUT_SIZE[2])
model.predict(img)


error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [289]:
X_train.shape

(96, 224, 224, 3)